# Basics of deep learning and neural networks


In [13]:
import numpy as np
from typing import NewType, Dict, List

In [2]:
NumpyArray = NewType('NumpyArray', np.ndarray)

## Section 1. Basics of deep learning and neural networks

We will code a **forward propagation** (prediction) for our first neural network.

Say that each data point is a customer. The first input is how many accounts they have, and the second input is how many children they have. The model will predict **how many transactions the user makes in the next year**. 

In [7]:
# Hard-code input data and weights
num_accounts: int = 3
num_children: int = 5

input_data: NumpyArray = np.array([num_accounts, num_children])
    
weights: Dict[str, NumpyArray] = {'node_0': np.array([2, 4]), 
                                  'node_1': np.array([ 4, -5]), 
                                  'output': np.array([2, 7])}

# Calculate the nodes and the output of the neural network
node_0_value: np.int32 = (input_data * weights['node_0']).sum()

node_1_value: np.int32 = (input_data * weights['node_1']).sum()

hidden_layer_outputs: NumpyArray = np.array([node_0_value, node_1_value])

output: np.int32 = (hidden_layer_outputs * weights['output']).sum()

output

-39

It looks like the network generated a prediction of $-39$ transaction. This number does not make sense as we have to add an activation function as in the next section.

## Section 2. The Rectified Linear Activation Function

An "activation function" is a function applied at each node. It converts the node's input into some output.

The rectified linear activation function (called **ReLU**) has been shown to lead to very high-performance networks. This function takes a single number as an input, returning 0 if the input is negative, and the input if the input is positive.

In [10]:
def relu(input):
    '''ReLU activation function.'''
    return max(input, 0)

In [11]:
node_0_input = (input_data * weights['node_0']).sum()
node_0_output = relu(node_0_input)

node_1_input = (input_data * weights['node_1']).sum()
node_1_output = relu(node_1_input)

hidden_layer_outputs = np.array([node_0_output, node_1_output])

model_output = (hidden_layer_outputs * weights['output']).sum()

model_output

52

We predicted $52$ transactions. Without this activation function, you would have predicted a negative number ($-39$)! The real power of activation functions will come soon when we start tuning model weights.

## Section 3. Applying the network to many observations/rows of data

You'll now define a function called `predict_with_network()` which will generate predictions for multiple data observations.

In [12]:
def predict_with_network(input_data_row, weights):

    node_0_input = (input_data_row * weights['node_0']).sum()
    node_0_output = relu(node_0_input)

    node_1_input = (input_data_row * weights['node_1']).sum()
    node_1_output = relu(node_1_input)

    hidden_layer_outputs: NumpyArray = np.array([node_0_output, node_1_output])

    input_to_final_layer = (hidden_layer_outputs * weights['output']).sum()
    model_output = relu(input_to_final_layer)

    return model_output

In [15]:
input_data: List[NumpyArray] = [np.array([3, 5]),
                                np.array([ 1, -1]),
                                np.array([0, 0]),
                                np.array([8, 4])]

weights: Dict[str, NumpyArray] = {'node_0': np.array([2, 4]),
                                  'node_1': np.array([ 4, -5]),
                                  'output': np.array([2, 7])}

In [16]:
results: List[np.int32] = []
for input_data_row in input_data:
    results.append(predict_with_network(input_data_row, weights))

results

[52, 63, 0, 148]


## Section 4. Deeper networks

An important fact about deep networks is that they internally build up representations of the patterns in the data that are useful for making predictions. And they find increasingly complex patterns as we go through successive hidden layers of the network.

Deep learning is also sometimes called **representation learning**, because subsequent layers build increasingly sophiscticated represnetations of the raw data, until we get to a stage where we can make preditions.

The cool thing about DL is that we do not need to specify the interactions. Instead, the network gets weights that find the relevant patterns to make better predictions.

In [21]:
def predict_with_network(input_data: NumpyArray, weights):
    """Predicts the number of transactions using a deep network."""
    # Calculate node 0 in the first hidden layer
    node_0_0_input = (input_data * weights['node_0_0']).sum()
    node_0_0_output = relu(node_0_0_input)

    # Calculate node 1 in the first hidden layer
    node_0_1_input = (input_data * weights['node_0_1']).sum()
    node_0_1_output = relu(node_0_1_input)

    # Put node values into a numpy array
    hidden_0_outputs: NumpyArray = np.array([node_0_0_output, node_0_1_output])
    
    # Calculate node 0 in the second hidden layer
    node_1_0_input = (hidden_0_outputs * weights['node_1_0']).sum()
    node_1_0_output = relu(node_1_0_input)

    # Calculate node 1 in the second hidden layer
    node_1_1_input = (hidden_0_outputs * weights['node_1_1']).sum()
    node_1_1_output = relu(node_1_1_input)

    # Put node values into a numpy array
    hidden_1_outputs = np.array([node_1_0_output, node_1_1_output])

    # Calculate model output
    model_output = (hidden_1_outputs * weights['output']).sum()
    
    return(model_output)

In [22]:
input_data: NumpyArray = np.array([3, 5])

weights: Dict[str, NumpyArray] = {'node_0_0': np.array([2, 4]),
                                  'node_0_1': np.array([ 4, -5]),
                                  'node_1_0': np.array([-1,  2]),
                                  'node_1_1': np.array([1, 2]),
                                  'output': np.array([2, 7])}
    
output = predict_with_network(input_data, weights)
print(output)

182


The network generated a prediction of 182 transactions.